In [182]:
import pandas as pd

# Названия колонок
column_full_names = [
    "Уровень преступности на душу населения по городам",
    "Доля жилой земли, отведенной под участки площадью более 25 000 кв. футов",
    "Доля акров нерозничного бизнеса по городам",
    "Фиктивная переменная реки Чарльз (1 если участок граничит с рекой; 0 в противном случае)",
    "Концентрация оксидов азота (частей на 10 миллионов)",
    "Среднее количество комнат в жилище",
    "Доля домов, занятых владельцами, построенных до 1940 года",
    "Взвешенные расстояния до пяти бостонских центров занятости",
    "Индекс доступности к радиальным highways",
    "Полная ставка налога на имущество за $10,000",
    "Соотношение учеников и учителей по городам",
    "1000(Bk - 0,63)², где Bk - доля чернокожего населения по городам",
    "процент населения с низким социально-экономическим статусом",
    "Медианная стоимость дома, занимаемого владельцeм в тысячах долларов ($1000)"
]

# Чтение CSV
df = pd.read_csv("housing.csv", delimiter=r"\s+", header=None, names=column_full_names)

# Преобразование в числовые списки
data = df.astype(float).values.tolist()  # ← Ключевой момент: .astype(float)

# Разделение на X и y
X = [row[:-1] for row in data]
y = [row[-1] for row in data]


In [183]:




def normalize(X):
    features = list(zip(*X))
    means = [sum(f)/len(f) for f in features]
    stds = [((sum((x - m)**2 for x in f)/len(f))**0.5) for f, m in zip(features, means)]
    X_norm = [[(x - m)/s if s != 0 else 0 for x, m, s in zip(row, means, stds)] for row in X]
    return X_norm

X = normalize(X)
for row in X:
    row.insert(0, 1.0)  # bias term

def train(X, y, lr=0.01, epochs=685):  # число эпох для демонстрации
    m = len(X)
    n = len(X[0])
    weights = [0.0] * n

    for epoch in range(epochs):
        predictions = [sum(w * x for w, x in zip(weights, row)) for row in X]
        errors = [pred - true for pred, true in zip(predictions, y)]

        gradients = [0.0] * n
        for j in range(n):
            gradients[j] = (2/m) * sum(errors[i] * X[i][j] for i in range(m))

        # Лог перед обновлением
        print(f"\nEpoch {epoch}")
        print("Weights:", ["{:.4f}".format(w) for w in weights])
        print("Gradients:", ["{:.4f}".format(g) for g in gradients])
        mse = sum(e**2 for e in errors) / m
        print("MSE:", round(mse, 4))

        # Обновление
        weights = [w - lr * g for w, g in zip(weights, gradients)]

    return weights

def predict(X, weights):
    return [sum(w * x for w, x in zip(weights, row)) for row in X]

weights = train(X, y)
y_pred = predict(X, weights)

# Финальная ошибка
final_mse = sum((yp - yt)**2 for yp, yt in zip(y_pred, y)) / len(y)
print("\nFinal MSE:", round(final_mse, 4))



Epoch 0
Weights: ['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']
Gradients: ['-45.0656', '7.1355', '-6.6236', '8.8889', '-3.2206', '7.8525', '-12.7780', '6.9269', '-4.5927', '7.0128', '8.6098', '9.3311', '-6.1277', '13.5553']
MSE: 592.1469

Epoch 1
Weights: ['0.4507', '-0.0714', '0.0662', '-0.0889', '0.0322', '-0.0785', '0.1278', '-0.0693', '0.0459', '-0.0701', '-0.0861', '-0.0933', '0.0613', '-0.1356']
Gradients: ['-44.1643', '6.2716', '-5.7650', '7.7255', '-3.1267', '6.7450', '-11.9465', '5.9118', '-3.5870', '5.9470', '7.4683', '8.5295', '-5.4325', '12.3980']
MSE: 563.4333

Epoch 2
Weights: ['0.8923', '-0.1341', '0.1239', '-0.1661', '0.0635', '-0.1460', '0.2472', '-0.1284', '0.0818', '-0.1296', '-0.1608', '-0.1786', '0.1156', '-0.2595']
Gradients: ['-43.2810', '5.5129', '-5.0132', '6.7065', '-3.0341', '5.7790', '-11.1986', '5.0261', '-2.7134', '5.0145', '6.4681', '7.8164', '-4.8221', '11.372

In [184]:
new_house = [
    0.2,   # Уровень преступности
    0.0,   # Доля земли >25k кв. футов
    7.07,  # Доля бизнеса
    0.0,   # Река Чарльз
    0.469, # NOx
    6.421, # Комнат
    78.9,  # Доля старых домов
    4.9671,# Расстояние до центров
    2.0,   # Индекс радиальных дорог
    242.0, # Налог
    17.8,  # Соотношение ученик/учитель
    396.9, # Bk
    9.14   # % бедного населения
]

def normalize_with_stats(X):
    features = list(zip(*X))
    means = [sum(f)/len(f) for f in features]
    stds = [((sum((x - m)**2 for x in f)/len(f))**0.5) for f, m in zip(features, means)]
    X_norm = [[(x - m)/s if s != 0 else 0 for x, m, s in zip(row, means, stds)] for row in X]
    return X_norm, means, stds

X_norm, means, stds = normalize_with_stats(X)

# Нормализуем new_house
new_house_norm = [(x - m)/s if s != 0 else 0 for x, m, s in zip(new_house, means, stds)]

new_house_norm.insert(0, 1.0)


In [185]:
def predict_one(x, weights):
    return sum(w * xi for w, xi in zip(weights, x))

predicted_price = predict_one(new_house_norm, weights)
print("Предсказанная стоимость дома ($1000):", round(predicted_price, 2))


Предсказанная стоимость дома ($1000): 25.16


In [186]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# 1. Загрузка данных
column_full_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE",
    "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

df = pd.read_csv("housing.csv", delimiter=r"\s+", header=None, names=column_full_names)

X = df.drop(columns=["MEDV"]).values
y = df["MEDV"].values

# 2. Нормализация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Обучение модели
model = LinearRegression()
model.fit(X_scaled, y)

# 4. Предсказание и оценка
y_pred = model.predict(X_scaled)
mse = mean_squared_error(y, y_pred)
print("Final MSE:", round(mse, 4))
print(model.coef_)


# 5. Нормализация новых данных
new_house_scaled = scaler.transform([new_house])

# 6. Предсказание
predicted_price = model.predict(new_house_scaled)[0]
print("Предсказанная стоимость дома ($1000):", round(predicted_price, 2))


Final MSE: 21.8948
[-0.92814606  1.08156863  0.1409      0.68173972 -2.05671827  2.67423017
  0.01946607 -3.10404426  2.66221764 -2.07678168 -2.06060666  0.84926842
 -3.74362713]
Предсказанная стоимость дома ($1000): 25.01


In [ ]:
import pandas as pd
import random
from math import sqrt

# Конфигурация
TARGET = "MEDV"
RANDOM_SEED = 42
TRAIN_RATIO = 0.8

# Пример колонок Boston Housing
column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE",
    "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

# 1) Загрузка
df = pd.read_csv("housing.csv", delimiter=r"\s+", header=None, names=column_names)

# 2) Очистка
df = df.drop_duplicates()
# Вариант А: удаляем пропуски. При необходимости — используй fillna стратегию.
df = df.dropna()
# Приведение к числовому типу
df = df.astype(float)

# 3) Стандартизация (сохранение статистик)
def standardize_df(df, exclude_cols):
    means = {}
    stds = {}
    df_std = df.copy()
    for col in df.columns:
        if col in exclude_cols:
            continue
        m = df[col].mean()
        s = df[col].std(ddof=0)  # как в формуле выше (делим на m, а не на m-1)
        means[col] = m
        stds[col] = s if s != 0 else 1.0
        df_std[col] = df[col].apply(lambda v: (v - m) / (stds[col]) if stds[col] != 0 else 0.0)
    return df_std, means, stds

df_std, feat_means, feat_stds = standardize_df(df, exclude_cols=[TARGET])

# 4) Корреляции с целью (Пирсон)
def pearson_corr(x_list, y_list):
    m = len(x_list)
    mean_x = sum(x_list) / m
    mean_y = sum(y_list) / m
    var_x = sum((xi - mean_x) ** 2 for xi in x_list) / m
    var_y = sum((yi - mean_y) ** 2 for yi in y_list) / m
    std_x = sqrt(var_x) if var_x > 0 else 0.0
    std_y = sqrt(var_y) if var_y > 0 else 0.0
    if std_x == 0 or std_y == 0:
        return 0.0
    cov = sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x_list, y_list)) / m
    return cov / (std_x * std_y)

correlations = {}
for col in df_std.columns:
    if col == TARGET:
        continue
    correlations[col] = round(pearson_corr(df_std[col].tolist(), df_std[TARGET].tolist()), 3)

print("Корреляции с MEDV (по модулю, убыв.):")
for k, v in sorted(correlations.items(), key=lambda kv: abs(kv[1]), reverse=True):
    print(f"{k}: {v}")

# 5) Разбиение на train/test
data = df_std.values.tolist()
random.Random(RANDOM_SEED).shuffle(data)
split = int(TRAIN_RATIO * len(data))
train_rows = data[:split]
test_rows = data[split:]

# Формируем X, y
feature_cols = [c for c in df_std.columns if c != TARGET]
def rows_to_Xy(rows):
    X = [[row[df_std.columns.get_loc(c)] for c in feature_cols] for row in rows]
    y = [row[df_std.columns.get_loc(TARGET)] for row in rows]
    # Добавляем bias в X
    for r in X:
        r.insert(0, 1.0)
    return X, y

X_train, y_train = rows_to_Xy(train_rows)
X_test, y_test = rows_to_Xy(test_rows)

# Предсказания и метрики
def predict_matrix(X, w):
    return [sum(wj * xj for wj, xj in zip(w, row)) for row in X]

def mse(y_true, y_pred):
    m = len(y_true)
    return sum((yp - yt) ** 2 for yp, yt in zip(y_pred, y_true)) / m

# Матрицы (списки списков): X^T X и X^T y
def matT_mat(A, B):
    # (A^T)B где A: m x n, B: m x k => A^T: n x m => результат n x k
    n = len(A[0])
    k = len(B[0])
    m = len(A)
    res = [[0.0]*k for _ in range(n)]
    for i in range(n):        # по строкам A^T (столбцам A)
        for j in range(k):    # по столбцам B
            s = 0.0
            for t in range(m):
                s += A[t][i] * B[t][j]
            res[i][j] = s
    return res

def matT_vec(A, y):
    # (A^T) y где A: m x n, y: m => результат n
    n = len(A[0])
    m = len(A)
    res = [0.0]*n
    for i in range(n):
        s = 0.0
        for t in range(m):
            s += A[t][i] * y[t]
        res[i] = s
    return res

def solve_linear_system_gauss(A, b):
    # Решает A w = b методом Гаусса с частичным выбором главного элемента
    n = len(A)
    # Формируем расширенную матрицу
    M = [A[i][:] + [b[i]] for i in range(n)]
    for col in range(n):
        # выбор главного элемента
        pivot = max(range(col, n), key=lambda r: abs(M[r][col]))
        if abs(M[pivot][col]) < 1e-12:
            raise ValueError("Матрица вырождена или почти вырождена; нормальные уравнения нестабильны.")
        # перестановка строк
        M[col], M[pivot] = M[pivot], M[col]
        # нормировка
        pivot_val = M[col][col]
        for j in range(col, n+1):
            M[col][j] /= pivot_val
        # обнуление в остальных строках
        for r in range(n):
            if r == col:
                continue
            factor = M[r][col]
            for j in range(col, n+1):
                M[r][j] -= factor * M[col][j]
    # считываем решение
    return [M[i][n] for i in range(n)]

def fit_normal_equations(X, y):
    # X: m x (n+1), y: m
    # A = X^T X  (размер (n+1)x(n+1))
    # b = X^T y  (размер (n+1))
    # Решаем A w = b
    # Для A используем матричное произведение через вспомогательные функции
    # Сформируем B = X (как B: m x (n+1)) и используем matT_mat
    B = X
    # A = X^T X
    # Сначала считаем X^T X как matT_mat(X, X), но нам нужен второй аргумент как матрица (m x (n+1))
    # X уже в таком формате; результат: (n+1) x (n+1)
    A = matT_mat(B, B)
    # b = X^T y
    b = matT_vec(X, y)
    # Решаем систему
    w = solve_linear_system_gauss(A, b)
    return w

w_normal = fit_normal_equations(X_train, y_train)
y_pred_ne_train = predict_matrix(X_train, w_normal)
y_pred_ne_test  = predict_matrix(X_test,  w_normal)
print("NormalEq MSE train:", round(mse(y_train, y_pred_ne_train), 4))
print("NormalEq MSE test :", round(mse(y_test,  y_pred_ne_test), 4))

def fit_gradient_descent(X, y, lr=0.01, epochs=1000):
    m, n = len(X), len(X[0])
    w = [0.0] * n
    for epoch in range(epochs):
        preds = predict_matrix(X, w)
        # ошибки
        errors = [p - t for p, t in zip(preds, y)]
        # градиенты по каждому w_j
        grads = [0.0] * n
        for j in range(n):
            s = 0.0
            for i in range(m):
                s += errors[i] * X[i][j]
            grads[j] = (2.0 / m) * s
        # обновление весов
        for j in range(n):
            w[j] -= lr * grads[j]
        # при желании можно логировать MSE
        # if (epoch+1) % 200 == 0:
        #     print(f"epoch {epoch+1} mse={mse(y, preds):.4f}")
    return w

w_gd = fit_gradient_descent(X_train, y_train, lr=0.01, epochs=2000)
y_pred_gd_train = predict_matrix(X_train, w_gd)
y_pred_gd_test  = predict_matrix(X_test,  w_gd)
print("GradDesc MSE train:", round(mse(y_train, y_pred_gd_train), 4))
print("GradDesc MSE test :", round(mse(y_test,  y_pred_gd_test), 4))


def standardize_new(sample_dict, means, stds, feature_order):
    # sample_dict: {feature_name: raw_value}
    # возвращает стандартизованный вектор признаков в порядке feature_order
    xs = []
    for col in feature_order:
        m = means[col]
        s = stds[col] if stds[col] != 0 else 1.0
        xs.append((sample_dict[col] - m) / s)
    return xs

# Пример нового дома
new_house = {
    "CRIM": 0.2,  "ZN": 0.0,   "INDUS": 7.07, "CHAS": 0.0, "NOX": 0.469,
    "RM": 6.421,  "AGE": 78.9, "DIS": 4.9671, "RAD": 2.0,  "TAX": 242.0,
    "PTRATIO": 17.8, "B": 396.9, "LSTAT": 9.14
}

# Стандартизируем и добавим bias
x_new_std = standardize_new(new_house, feat_means, feat_stds, feature_cols)
x_new_std_bias = [1.0] + x_new_std

# Предсказания двумя моделями
y_new_ne = sum(wj * xj for wj, xj in zip(w_normal, x_new_std_bias))
y_new_gd = sum(wj * xj for wj, xj in zip(w_gd,     x_new_std_bias))
print("Pred (NormalEq) new:", round(y_new_ne, 3))
print("Pred (GradDesc) new:", round(y_new_gd, 3))


Корреляции с MEDV (по модулю, убыв.):
LSTAT: -0.738
RM: 0.695
PTRATIO: -0.508
INDUS: -0.484
TAX: -0.469
NOX: -0.427
CRIM: -0.388
RAD: -0.382
AGE: -0.377
ZN: 0.36
B: 0.333
DIS: 0.25
CHAS: 0.175
NormalEq MSE train: 22.3225
NormalEq MSE test : 21.9513
GradDesc MSE train: 22.3273
GradDesc MSE test : 21.9423
Pred (NormalEq) new: 25.215
Pred (GradDesc) new: 25.196
